# Algoritmos Genéticos

In [17]:
import random

## Clase Individuos

- **generate_random_array:** generación de cada individuo con **a** y **j** en la primera y última posición, generándose de forma aleatoria los otros 6 valores para tener cada individuo con 8 cromosomas
- **fitness_function:** aplicación de la **función de calidad** a cada cromosoma del individuo y se aplica la función inversa a cada uno (Se realiza una multiplicación con los adyacentes)
- **function_adition:** suma de la función y la función inversa de cada cromosoma para obtener el total correspondiente al individuo

In [180]:
class Individuos():
    def __init__(self, fichas):
        self.fichas = fichas
        self.individuo = []
        self.function = []
        self.inverse_function = []
        self.total_function = 0
        self.total_inverse_function = 0
        self.probabilidad = 0
        self.probabilidad_inversa = 0
        self.probabilidad_acumulada = []

    def generate_random_array(self):
        random_array = [random.randint(2, 9) for _ in range(6)]
        self.individuo.append(fichas[list(fichas.keys())[0]])
        for i in range(6):
            self.individuo.append(fichas[random.choice(list(fichas.keys()))])
        self.individuo.append(fichas[list(fichas.keys())[-1]])
    
    def fitness_function(self):
        for i in range(len(self.individuo)):
            if i == 0:
                self.function.append(self.individuo[i] * self.individuo[i + 1])
            elif i == len(self.individuo) - 1:
                self.function.append(self.individuo[i] * self.individuo[i - 1])
            else:
                self.function.append(self.individuo[i] * self.individuo[i - 1] * self.individuo[i + 1])
            self.inverse_function.append(1 / self.function[i])
        
        for value in self.function:
            self.inverse_function.append(1 / value)
    
    def function_addition(self):
        for i in range(len(self.function)):
            self.total_function += self.function[i]
        self.total_inverse_function = 1/self.total_function
            

### Fichas

In [138]:
fichas = {
    "a": 10,
    "b": 3,
    "c": 7,
    "d": 1,
    "e": 2,
    "f": 8,
    "g": 4,
    "h": 10,
    "i": 9,
    "j": 3
}

### Implementación

In [170]:
individuo = Individuos(fichas)
individuo.generate_random_array()
individuo.fitness_function()
individuo.function_addition()

print("Individuo:", individuo.individuo)
print("Fitness Function:", individuo.function)
print("Inverse Function:", individuo.inverse_function)
print("Total Function:", individuo.total_function)
print("Total Inverse Function:", individuo.total_inverse_function)

Individuo: [10, 8, 3, 2, 10, 8, 7, 3]
Fitness Function: [80, 240, 48, 60, 160, 560, 168, 21]
Inverse Function: [0.0125, 0.004166666666666667, 0.020833333333333332, 0.016666666666666666, 0.00625, 0.0017857142857142857, 0.005952380952380952, 0.047619047619047616, 0.0125, 0.004166666666666667, 0.020833333333333332, 0.016666666666666666, 0.00625, 0.0017857142857142857, 0.005952380952380952, 0.047619047619047616]
Total Function: 1337
Total Inverse Function: 0.0007479431563201197


## Creación de población y probabilidades

- **creacion_poblacion:** se genera una población (array) con la cantidad de individuos que se asignen
- **probabilidades:** se calcula la probabilidad (total) de cada individuo, utilizando (1/f)/sum(f)

In [181]:
def creacion_poblacion(fichas, cantidad):
    poblacion = []
    for _ in range(cantidad):
        individuo = Individuos(fichas)
        individuo.generate_random_array()
        individuo.fitness_function()
        individuo.function_addition()
        poblacion.append(individuo)
    return poblacion

def probabilidades(poblacion):
    for individuo in poblacion:
        individuo.probabilidad = individuo.total_inverse_function / sum(ind.total_inverse_function for ind in poblacion)

def probabilidades_acum(poblacion):
    acumulada = 0
    for individuo in poblacion:
        acumulada += individuo.probabilidad
        individuo.probabilidad_acumulada.append(acumulada)

# def probabilidades_inversa(poblacion):
#     for individuo in poblacion:
#         individuo.probabilidad_inversa = individuo.total_inverse_function / sum(ind.total_inverse_function for ind in poblacion)

### Implementación

In [256]:
poblacion = creacion_poblacion(fichas, 5)
probabilidades(poblacion)
probabilidades_acum(poblacion)

print("Probabilidades Acumuladas:")
for ind in poblacion:
    print(f"Individuo: {ind.individuo}, Probabilidad: {ind.probabilidad}, Probabilidad Acumulada: {ind.probabilidad_acumulada}")

print("Total Probabilidad:", sum(ind.probabilidad for ind in poblacion))

Probabilidades Acumuladas:
Individuo: [10, 9, 7, 7, 7, 10, 1, 3], Probabilidad: 0.08146792459522655, Probabilidad Acumulada: [0.08146792459522655]
Individuo: [10, 3, 10, 3, 2, 10, 3, 3], Probabilidad: 0.24440377378567962, Probabilidad Acumulada: [0.3258716983809062]
Individuo: [10, 9, 2, 3, 4, 10, 3, 3], Probabilidad: 0.2486728353365212, Probabilidad Acumulada: [0.5745445337174274]
Individuo: [10, 8, 3, 4, 7, 1, 7, 3], Probabilidad: 0.27599069123778686, Probabilidad Acumulada: [0.8505352249552143]
Individuo: [10, 3, 3, 7, 10, 9, 1, 3], Probabilidad: 0.1494647750447857, Probabilidad Acumulada: [1.0]
Total Probabilidad: 0.9999999999999999


## Selección

- El individuo con mayor función de calidad obtiene mayor probabilidad de ser seleccionado en el método de la ruleta (Se utiliza 1/f debido a que en el problema se está intentando minimizar)
- Se deben seleccionar 2 individuos para realizar mutación o cruce

In [263]:
def wheel_selection(poblacion, exclude=None):
    random_value = random.random()
    print("Random Value:", random_value)
    for ind in poblacion:
        if random_value <= ind.probabilidad_acumulada[-1]:
            return ind
    return poblacion[-1]

individuo1 = wheel_selection(poblacion)
individuo2 = wheel_selection(poblacion, exclude=individuo1)
print("Selected Individual 1:", individuo1.individuo, "with Probability:", individuo1.probabilidad)
print("Selected Individual 2:", individuo2.individuo, "with Probability:", individuo2.probabilidad)

Random Value: 0.46683850629445034
Random Value: 0.3649293524989152
Selected Individual 1: [10, 9, 2, 3, 4, 10, 3, 3] with Probability: 0.2486728353365212
Selected Individual 2: [10, 9, 2, 3, 4, 10, 3, 3] with Probability: 0.2486728353365212


## Cruces

In [205]:
def crossover(ind1, ind2):
    crossover_point = random.randint(1, len(ind1.individuo) - 2)
    child1 = ind1.individuo[:crossover_point] + ind2.individuo[crossover_point:]
    child2 = ind2.individuo[:crossover_point] + ind1.individuo[crossover_point:]
    return child1, child2

child1, child2 = crossover(individuo1, individuo2)
print("Child 1:", child1)
print("Child 2:", child2)

Child 1: [10, 3, 1, 10, 4, 3, 3, 3]
Child 2: [10, 3, 1, 10, 4, 3, 3, 3]


## Mutaciones